In [1]:
import json

import numpy as np
np.random.seed(42)

import spacy

In [2]:
!pip3 install nltk

In [3]:
!pip3 install pymystem3

In [4]:
!pip3 install natasha

In [5]:
# download stopwords corpus, you need to run it once
import nltk
# nltk.download("stopwords")
# #--------#

In [6]:
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

In [7]:
#Create lemmatizer and stopwords list
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

In [8]:
#Preprocess function
def tokenize(text: str) -> list:
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    #text = " ".join(tokens)
    
    return tokens

In [9]:
#Examples    
print(tokenize("Ну что сказать, я вижу кто-то наступил на грабли, Ты разочаровал меня, ты был натравлен."))
#> 'сказать видеть кто-то наступать грабли разочаровывать натравлять'

print(tokenize("По асфальту мимо цемента, Избегая зевак под аплодисменты. Обитатели спальных аррондисманов"))
#> 'асфальт мимо цемент избегать зевака аплодисменты обитатель спальный аррондисман'

['сказать', 'видеть', 'кто-то', 'наступать', 'грабли', 'разочаровывать', 'натравлять']
['асфальт', 'мимо', 'цемент', 'избегать', 'зевака', 'аплодисменты', 'обитатель', 'спальный', 'аррондисман']


In [10]:
# thanks DS community: https://www.kaggle.com/code/alxmamaev/how-to-easy-preprocess-russian-text/script

In [11]:
import logging

In [12]:
def log(print_text: str):
    print(print_text)

In [434]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from razdel import sentenize


class LuhnSummarizer():
    def __init__(self, is_lemmatize: bool = True):
        self.is_lemmatize = True
        self.sf_word_threshold = 0.002
        self.sf_sentence_threshold = 0.3
        self.max_not_sf_seq_words = 4

    def custom_sentenize(self, text: str) -> list:
        sents = []
        for line in text.splitlines():
            line = line.lower() if line.strip().endswith("\n") else line
            sents += [sent.text for sent in sentenize(line) if sent.text != ""]

        sent_char_lst = []
        char_num = 0
        for sent_num, sent in enumerate(sents):
            while char_num < len(text):
                if text[char_num:char_num + len(sent)] == sent:
                    # Sentence found
                    new_sent = text[char_num:char_num + len(sent)]
                    sent_char_lst.append(new_sent)
                    char_num += len(new_sent)
                    break
                else:
                    sent_char_lst.append(text[char_num])
                    char_num += 1
        # remaining characters
        if char_num < len(text):
            sent_char_lst.append(text[char_num:])

        return [x for x in sent_char_lst if x != " "]

    def tokenize_sent(self, sentences: list) -> list:

        # tokenization
        tokens = [tokenize(sent) for sent in sentences]  # [x for sent in sentences for x in tokenize(sent)]

        return tokens

    def create_word_freq_dict(self, text: str) -> dict:
        # log(f"tokens:{tokens}")

        tokens = tokenize(text)
        vectorizer = TfidfVectorizer(use_idf=False)
        X = vectorizer.fit_transform(tokens)
        features_names_out = vectorizer.get_feature_names_out()

        freq_df = pd.DataFrame(X.toarray(), columns=features_names_out)  # составляем датафрейм частотностей слов
        #print(f"Features Names Out: {features_names_out}")
              
        freq_dict = {word: np.sum(freq_df[word].values) / len(list(freq_df.keys())) for word in features_names_out}

        freq_dict = dict(sorted(freq_dict.items(), key=lambda y: y[1], reverse=True)) # сортируем словарь частотностей слов по убыванию
        #print(f"Freq Dict before split by threshold: {freq_dict}")
        freq_dict = {k:v for k,v in freq_dict.items() if v >= self.sf_word_threshold}

        return freq_dict

    def get_sentence_significance_word_mask(self, sentence_words_mask: list) -> list:   
        first_sf_word_indx = sentence_words_mask.index(1)
        last_sf_word_indx = len(sentence_words_mask) - 1 - sentence_words_mask[::-1].index(1)

        return sentence_words_mask[first_sf_word_indx: last_sf_word_indx + 1]


    def compute_significance_factor(self, freq_dict: dict, sentence: list) -> np.float16:
        sentence_words_mask = [1 if freq_dict[word]  else 0 \
                                                          for word in sentence if word in freq_dict.keys()]
        # example of sentence_words_mask: [0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0] 1 - sf word in sentence and 0 - not sf word
        # print(sentence_words_mask)
        
        if sum(sentence_words_mask) == 0:
            return 0
        
        sentence_words_mask = self.get_sentence_significance_word_mask(sentence_words_mask)
        number_of_sf_words = sum(sentence_words_mask)
        total_number_of_bracketed_words = len(sentence_words_mask)
        
        
        #print(f"number_of_sf_words: {number_of_sf_words}, total_number_of_bracketed_words: {total_number_of_bracketed_words}")
        significance_factor = number_of_sf_words ** 2 / total_number_of_bracketed_words

        return significance_factor


    def summarize(self, text: str) -> str:
        # sentence segmentation
        sentences = self.custom_sentenize(text)
        #print(len(sentences))
        # log(sentences)

        text_freq_dict = self.create_word_freq_dict(text)
        #log(text_freq_dict)

        sentences_sf = []
        for sent in sentences:
            sentence_tokens = tokenize(sent)
            # log(sentence_tokens)

            sentences_sf.append(
                self.compute_significance_factor(text_freq_dict, sentence_tokens) if len(sentence_tokens) > 0 else 0)
        # log(sentences_sf)

        sentences_sf.sort(reverse=True)

        sentence_sf_threshold_percentile_75 = np.percentile(sentences_sf, 75)
        #print(sentences_sf)
        #print(sentences_sf[ranking_ind_uppper_bound])
        # log(min(sentences_sf[:ranking_ind_uppper_bound]))

        summary_sentences = [sent for i, sent in enumerate(sentences) if
                             sentences_sf[i] > sentence_sf_threshold_percentile_75]

        return "".join(summary_sentences)

In [435]:
summarizer = LuhnSummarizer()

summarizer.summarize("Вася привет")

''

In [436]:
summarizer = LuhnSummarizer()

summarizer.summarize("Вася привет, надо написать отчет для привета. \n С уважением вася вася вася вася вася вася")

'Вася привет, надо написать отчет для привета.'

In [437]:
test_article = """
Два года назад я написал на хабре статью Как живется в США «неайтишникам», которая собрала 194 000 просмотров, 561 комментарий и вошла в список лучших статей 2020 года. И вот, два года спустя я решил пробежаться по тем же тезисам, чтобы провести некий срез, как изменилась обстановка в США и мое место в ней, особенно в свете последних событий. Обещаю без политики и максимально объективно.
!!! Все что я тут пишу это исключительно мой опыт и мое мнение. Я не претендую на истину в последней инстанции, а просто рассказываю как это было у меня.
Про мой язык
За два прошедших года мой Английский однозначно улучшился. Точнее даже не сам язык, а способность им пользоваться. Ушла закрепощенность и ощущение убогости. Теперь даже если я говорю неправильно, меня это не смущает. Причем все это в основном благодаря работе, где приходится очень много коммуницировать и читать огромное количество документации. Но тут же есть и обратная сторона. Окружающие меня американцы довольно свободно относятся к правилам грамматики. Легко меняют времена, лица, окончания. Используют много жаргонизмов. Иногда даже спрашивают меня как правильно пишется то или иное слово. Это, конечно, забавно.
Моя работа в США
Как я уже писал в предыдущей статье, после работы верхолазом на вышках сотовой связи (видео как я залажу на 400 футовую мачту) я перешел на работу по специальности в местную фармкомпанию, которая занимается контрактным производством лекарств находящихся на разных фазах испытаний. То есть заказчик нам присылает готовую клеточную культуру с определенным геномом, способную продуцировать нужный продукт, а мы, следуя инструкциям и требованиям ее растим в биореакторах 25л - 50л - 200л - 500л - 2000л, попутно отбирая тестовые образцы, а потом выделяем продукт. Поскольку в основе чаще всего используются культуры CHO,то и условия роста и кормления примерно одинаковые. То есть работа в целом не бей лежачего.
Карьерный путь выглядит так: Manufacturing Associate I - Manufacturing Associate II - Senior Manufacturing Associate I - Senior Manufacturing Associate II - Team Lead - Supervisor. Между ступеньками должно пройти минимум 18 мес, это при условии что ты очень хорошо владеешь языком (это не в смысле знаний английского). Зарплата начинается с 20$ в час и на должности супервайзера получается около 40$ в час. Плюс индексация 2,5-5% в год, плюс 401К. Налоги составляют в среднем 17-20%.
Картинка из интернета, но мое рабочее место выглядит один в один.

При этом меня поражает уровень образования работников. Скажем человек с бакалаврской степенью спокойно может не знать как перевести граммы в килограммы. Величину 3,5 часа спокойно трактуют как 3 часа 50 минут. Или что если в закрытый сосуд закачивать воду, то воздух, находящийся в сосуде полностью растворится в воде и весь объем сосуда займет вода. Или, что если оставить электроплитку включенной, она нагреется не до максимальной температуры установленной производителем, а будет расти бесконечно пока не расплавится (привет инженерам из отдела расследований).
На работе не в коем случае нельзя спорить с коллегами (я уже не говорю ругаться). Это харассмент. Если у тебя есть любое непонимание, иди к супервайзеру. И упаси тебя Бог назвать кого-то "не тимплеер". Это жутчайшее оскорбление. Даже несмотря на то, что он может реально быть таким.
Эффективность работников очень низкая. Я сначала удивлялся, когда люди увольнялись с формулировкой "сильно тяжело работать". Это при том что ты 30-40% времени реально что-то делаешь, а остальное время втыкаешь в ютубчик. Однажды уволилось сразу 5 человек из 8. Мы втроем, работая в обычном режиме спокойно вывозили следующие пару месяцев пока не набрали новых. Тут, кстати, тоже фишка. Первые 5-6 месяцев тебя вообще не трогают. Сиди читай документы, проходи тренинги. Многие так и поступают, пол-года сидят, получают зарплату, потом переходят в другую компанию на должность повыше.
Все, начиная от коллег, заканчивая директором, очень боятся ответственности. Любые решения могут приниматься месяцами. Все друг другу улыбаются, посылают "сенкью леттер", собирают митинги и презентации, но при этом не меняется вообще ничего. Например простая опечатка в процедуре, которая сбивает с толку исполнителя может исправляться годами. При этом ежемесячно невнимательный исполнитель на этом месте может совершать ошибку, требующую отдельного внутреннего расследования. Если рабочий день до 18-00, то ровно минута в минуту народ встает и идет домой. Прямо посреди процесса. Дверь на клюшку и пусть следующая смена разгребает.
Очень строгое разделение труда. Если я начал работать, а у прибора, скажем разболталась вилка и нужно просто взять отвертку и сделать пол-оборота винта - создается IT-тикет специальной команде, которая должна придти в течение 2-3 дней и починить. У меня обычно не хватает терпения ждать и легкие поломки я ремонтирую сам. За что имею очень большое уважение у коллег как "суперинженер". Ну и сам подход к работе интересен. Если написано, что нужно нажать вот эту кнопку, никто не будет даже думать зачем, и почему. Написано в процедуре - нажимаю. И даже если это очевидная опечатка, и даже если нажатие кнопки в текущий момент приведет к проблеме - пофигу, так написано. Примерно половина наших "факапов" происходит именно по этим причинам.
Это двухнедельный пэйчек:

Последние пол-года я хожу по собеседованиям и ищу другую работу. С мастерской степенью и 15 годами опыта хочется делать что-то более интересное. Но пока дела идут туго. Во первых, заполнение анкет на сайтах работодателей это просто отдельное ублюдство утомительно. Такое ощущение что ты попал в 1992 год с плывущими верстками, неработающими кнопками, и миллионом повторяющихся вопросов. Чтобы качественно заполнить одну анкету спокойно уходит 30-40 минут. Во вторых, если у тебя таки хватило терпения на все это, то ты попадаешь на телефонное интервью, где рекрутер, часто не имеющий представления об отрасли делает твой первичный отбор. Потом стандартное 3-5 часовое интервью со всеми - с директором, менеджером, супервайзером и даже потенциальными коллегами. С каждым по 30-40 минут. И за это время НИ ОДНОГО вопроса по процессу, по оборудованию, по опыту. Все вопросы из софтскилл - как решаешь конфликты, твой самый большой факап, что будешь делать если твой коллега сворует карандаш и т.д. В моей золотой коллекции есть такие - какой ты фрукт (должен быть тот что растет в гроздьях, иначе ты не тим плеер), кто ты из кухонного оборудования (хз что тут -должно быть).
Стоимость жизни в США в 2022 (в сравнении с 2020)
Аппартаменты с учетом вывоза мусора и воды минимум 1300$ в месяц. Два года назад было 950$. Рост 40%.
Электричество 40-80$ в месяц. Осталось примерно также, с учетом что мои новые аппарты на треть меньше по размеру.
Интернет 50$ в месяц. Осталось также.
Стрижка раз в месяц в дешевой парикмахерской 20$ (было 15$).
Мобильный телефон 15 $ плюс 10 Гб в месяц с учетом оплаты за год вперед. Особенно не поменялось. На двоих это, соответственно 30$.
Автостраховка 50$ в месяц. Не изменилось.
Бензин 200-300$ в месяц За два года стоимость бензина выросла почти в 2.5 раза, с ~1.9$ за галлон самого дешевого бензина (тут это 87) до ~4,5$.
Еда 600-800$ в месяц (было 300-400$). Но тут у всех по разному
Медицинская страховка мне обходится ежемесячно в 500$ из кармана, плюс еще столько же добавляет компания.
Итого, общая минимально необходимая сумма приближается к 3000$. То есть стоимость жизни за два года выросла почти в два раза (расскажите мне про огромную инфляцию в России). И если бы я переехал в США не тогда, а теперь, я бы очень быстро взгрустнул работая в доставке.
И это я еще не считаю детей. Детсад легко может стоить 60$ в день. Частная школа 1000$ в месяц. Муниципальные тоже есть, но там есть банды, и там, бывает, стреляют.
Как я не купил дом
Когда я только приехал в США, стоимость среднего дома 1980-2000 года постройки из говна и палок из бруса и фанеры в зависимости от локации составляла 150-200к. Я даже успел пару посмотреть. Но тогда у меня не было кредитной истории и дом мне не светил. Сейчас любой банк мне спокойно может дать 300к в ипотеку на 30 лет под 3,5-5% годовых, но указанные выше дома уже стоят 350-500к. То есть выросли в два раза за два года. При этом средняя экспозиция дома на рынке составляет 2-4 дня. И на просмотр спокойно может приехать 40 человек. И помимо указанной цены каждый втихаря начнет предлагать продавцу наличку в размере 15к-50к. В текущих условиях с моими 300к мне ловить просто нечего.
Таки пойти в IT
Как я писал раньше, каждый мигрант идет в IT, тестировщиком. Как когда-то во времена золотой лихорадки зарабатывали на лопатах, так сейчас зарабатывают на курсах. От бесплатных, трехдневных, до полугодовых за 12к.
Естественно у каждого "идущего" есть друг друга сестры подруги, который закончил двухнедельные курсы и его взяли тестировщиком в Фейсбук за 150к в год. Но пока идет учеба, идущие в IT работают трак-драйверами, верхолазами, водителями такси и т.д. Я лично не знаю ни одного "идущего", кто за прошедшие два года куда-то бы дошел.
Успешный проект на Кикстартере
Живя в США сложно было удержаться чтобы не попробовать запустить что-нибудь на Кикстартере. Задача была сделать что-то очень простое и дешевое, но интересное. И чтобы моих компетенций хватило полностью на весь проект. В голову пришла идея носимого девайса для хранения паролей. Это звучит сложно, на самом деле это просто пластиковая карточка форм-фактора банковской карты, на которой нанесены две группы случайных символов. Чтобы запомнить, а потом воспроизвести пароль нужно только лишь запомнить визуальный паттерн. Идея совсем не новая, но для пробы сойдет.

Возможно я потом напишу отдельную статью про этот проект на кикстартере, там много интересного и не очень. Но если коротко, очень важно в первые часы максимально возможно запостить ссылку везде, где только можно. Плюс к этому важно чтобы в первые часы произошло как можно больше донатов, тогда Кикстартер видит, что проект интересен публике, и начинает его показывать на главной. И огромное число околокикстартеровских "чаек", которые налетают и наперебой пытаются тебя раскрутить на покупку у них стратегии продвижения твоего продукта. Естественно без гарантий.
Напечатать карты в США оказалось невозможным. Я лично общался я с десятком типографий. Поскольку изначально цели заработать не было, я готов был платить даже из кармана до 5$ за карту и при этом полностью самостоятельно подготовить все макеты для печати по образцу подрядчика. Поскольку каждая карта подразумевалась уникальной, подразумевалось много ручной работы. Ни одна печатня даже не стала обсуждать. Ответ был один - "Мы так не делаем". Это к слову об эффективности бизнеса. В итоге, без лишних вопросов напечатали у замечательных ребят в Питере и DHL-ем отправили в США.
Финансы проекта. За месяц собрано около 9000$, потрачено на изготовление, доставку и рассылку около 3000$. В принципе, если проект простой в изготовлении и дешевый в отправке, Кикстартер вполне рабочая схема. Есть еще куча идей, что можно туда разместить.
Успешная книга на Амазоне
Живя в США сложно было удержаться чтобы не попробовать запустить что-нибудь на Амазоне. Там есть площадка, на которой можно размещать свои книги. Идея в том, что ты размещаешь PDF- версию и описание, а если кто-то эту книгу покупает, Амазон ее печатает и отсылает покупателю.

Писатель из меня никакой, поэтому для эксперимента я нарисовал две раскраски для детей. Цену поставил 5$. По условиям Амазона с этих пяти долларов я имел 1$. Итого одна раскраска мне приносит примерно 150-200 долл в год. Вторая ни разу не продалась. Теоретически, если нарисовать 100 книг, то можно, наверное, иметь 15к-20к пассивного дохода в год, но проверять это как-то лениво, поэтому оставлю идею до пенсии.
Кстати о Кикстартере, Амазоне, Etsy и т.д.
Если у вас есть вопросы по указанным площадкам, или есть интересные, подъемные идеи, пишите, чем смогу помогу.
Соотечественники в США
На всякий случай повторюсь, изложенное ниже это исключительно мой опыт и мое восприятие действительности. Так вот, как я вижу, мигранты в первом поколении никогда не будут чувствовать себя тут как дома. Их дети - да. И это четко видно невооруженным взглядом. Тут постоянно проводятся всякие встречи соотечественников с костром и гитарой. В местных сообществах ищут русскоязычных нянь, механиков, врачей, дантистов, адвокатов. Практически все кто прожил тут более 10, кого я знаю лично, так или иначе строят планы на возвращение. Кто-то ждет пока дети закончат школу и встанут на ноги, кто-то ждет пенсии, кто-то копит определенную сумму на счету, кто-то на заработанные тут деньги покупает землю и квартиры в России, кто-то вкладывает в Российский фондовый рынок. Особенно сложно стало теперь.У всех есть родители или родственники "там". Перевезти их сюда сейчас практически невозможно. Да какой там перевезти, просто слетать и то сложно.
Весь "наш" бизнес направлен исключительно на нашу же аудиторию, начиная с выпекания русских пирожков и учебы езды на велосипеде и заканчивая риэлторами, бровистами, отделочниками, автомеханиками, нянями и т.д. Заметили что в этом перечне нет никого выше технического персонала? Такова реальность. Только единицы могут организовать бизнес для американской аудитории.
Про текущее
Самый сложный для меня абзац, так как нужно подобрать максимально нейтральные слова. Поскольку тут довольно большая украинская диаспора, февраль 2022 года привнес в сообщество мигрантов ненависть. Украинцы проклинают русских, русские отвечают, что сами виноваты. За пределы фейсбуков это, слава Богу, не выходит. И тех и других понять можно. По крайней мере, у обеих сторон есть свои аргументы. Местные СМИ показывают исключительно и только проукраинскую точку зрения. Причем везде, где только возможно, выкуплена даже реклама в ютубе.
Американцы, которые окружают меня, слабо представляют что это и где это. Частые вопросы последнее время это "А что Украина это не Россия?", "А что в России уже не коммунизм?", "А что у украинцев другой язык?" и т.д. Власть пользуясь случаем все экономические сложности валит на Россию. Местных это объяснение удовлетворяет.
Многие компании, спортзалы, магазины проводят сбор денег для Украины. Каждый торопится высказать свою антироссийскую позицию. Даже моя компания разослала всем сотрудникам письмо, что не будет работать с Россией. Другой вопрос, что за два года я ни разу ни пол-слова не слышал что мы с ней вообще когда-либо работали. Насколько я понимаю, в России вообще нет подобных производств.
После февральских событий у меня было три интервью в разных компаниях, на должности, которым я точно подходил.В двух случаях у меня был референс от уже работающих там. Дальше телефонного интервью не прошло. В одном случае вообще после того как я сказал что из России, рекрутер плавно свернула беседу и попрощалась словами "мы вам обязательно перезвоним". Я не утверждаю, что это причина, но есть ощущение что не без этого. Знакомые американцы подтвердили мое опасение, сказали что это возможно.
Про свободу
Многие недовольны пропиской, что необходима в России. Да, в США прописки нет. Но! При смене адреса ты обязан: поменять права, так как в них указан адрес, иначе штраф; сделать уведомление на сайте налоговой; сделать уведомление на сайте миграционки; поменять адрес во всех банковских аккаунтах (иначе карты перестанут работать) и аккаунтах других служб; поменять адрес в государственной почте и т.д. И попробуй что-то пропустить.
Свобода слова безмерная. Особенно если это не касается темнокожих, женщин, ЛГБТ, всяких веганов, работодателя и иногда властей.
Все не любят всех. Белые черных, черные белых. Нормальные не любят ЛГБТ, и наоборот. Демократы - республиканцев. Все вместе опасаются русских и китайцев. Работяги не любят правительство. Это не плавильный котел, это слоеный пирог. И это всё существует вместе потому что пока нет недостатка в деньгах.
Про лицемерие и красивую обертку
То что изначально казалось вежливостью, на самом деле является лицемерием. Это окружает тебя везде, в магазинах, в банках, на работе. Соседи и продавцы тебе говорят что рады видеть, рекрутер тянет "вааааууу" читая твое резюме, коллеги делают круглые глаза, узнав что ты приехал с другой страны. Сначала ты воспринимаешь все всерьез. А потом понимаешь что за этим ничего нет.
Они на каждом углу кричат об экологии и сортируют мусор, но при этом три раза в день заказывают еду в пенопластовых контейнерах и два раза в день кофе в старбаксе с пластиковой крышкой и стоппером. Они требуют чтобы ты приходил на собеседование в костюме, даже на позицию сварщика, потому что это бизнес стайл, но при этом у тебя может быть полкилограмма пирсинга на лице, синие волосы и татуировка на лбу. Они каждое утро идут бегать, потому что того требует образ, а потом, прямо после бега идут в Старбакс пить кофе, чтобы проснуться. Они переживают за Украину, но две трети не сможет ее показать на карте. Они никогда не пропустят тебя на дороге, если тебе, например, неудобно повернуть или съехать. Если кто-то тебя поцарапает на парковке, то вероятнее всего он убежит. Ты можешь договориться на бизнес встречу, а на нее не придут, и не перезвонят. И так далее, и тому подобное.
Даже улыбки. Поскольку дантист тут многим не по карману, люди покупают "набор юного стоматолога" в волмарте. Это штука, типа боксерской капы, но спереди выглядит как идеальные зубы. И таких улыбок вокруг много )))
Про уровень сервиса
Про уровень сервиса написано и показано уже много. Многим знакома фраза "где открывали карту туда и идите". Только в США это устроено даже хуже. Ни одно их банковское приложение даже вполовину не дотягивает до наших. Кто пользовался знает. Сотрудники часто некомпетентные, причем решение проблемы выглядит как дозвон сотрудником на горячую линию своего же банка, ровно как и ты сам можешь сделать. Однажды я пять(!) раз приходил в ВеллсФарго банк чтобы разблокировать новую карту, которая заблокировалась на первой же покупке. Естественно, каждый раз я брал аппойнтмент. За пять визитов они так и не смогли ее разблокировать. Или Сити-банк. Покупал с их карты ноутбук. Потратил ровно день на телефоне, потому что платеж не проходил и никто не понимал почему. Или банк Чейз. Выпустили мне две карты и перепутали их реквизиты. Пришлось ехать в соседний город лично, прямо к главному менеджеру чтобы решить вопрос.
Невозможно никуда попасть без записи. Автомастерская, парикмахерская, банк, больница. Да что угодно. Это в России можно по пути на работу заскочить в сбер и, скажем, заказать карту. Но не тут.
Про здравоохранение вообще писать не хочу. За свои 1000$ в месяц я не могу просто попасть к врачу. Сначала нужно найти клинику которая принимает пациентов, и записаться на прием к терапевту. Это все происходит на вебсайтах, которые работают 50/50. И сто процентов это будет не сегодня и не завтра. Терапевт отправит к требуемому врачу, но там тоже нужно записаться, причем им нельзя позвонить, нужно ждать пока они сами тебе позвонят. Пройти УЗИ в день обращения невозможно. Ну и т.д. Я с теплотой вспоминаю родной провинциальный город в Сибири, где за цену чашки кофе можно в день обращения попасть к узкому специалисту и решить все свои проблемы.
Про зубы и глаза даже писать не буду.
Про то что в США лучше чем в России
По ходу статьи может сложиться впечатление, что я описываю только негативную сторону жизни в США. Для уравновешивания я хочу рассказать про то, что мне тут нравится, и чему нам стоило бы поучиться и перенять:
Качество жизни. Снимая аппарты за 1300$ я имею в полушаговой доступности два теннисных корта, бассейн, спортзал, огромное озеро с дорожками вокруг. И все это находится в зеленом лесу, и все это бесплатно.
Публичный спорт в целом. Тут огромное количество бесплатных кортов, баскетбольных площадок, парков, трейлов и велодорожек. Полно дешевых фитнес-клубов (10-20$ в мес), а работодатель часть их еще и оплачивает.
Уважение к произведенному в США. Если какая-то вещь сделана в США, будьте уверены, вы об этом узнаете.Это будет выбито в металле на самом видном месте. И люди этим гордятся. Нужно ли напоминать наше отношение к своему производству?
Уважение к своим солдатам. Не важно где они воюют. Не важно насколько демократические ценности они несут. Это вопросы к политикам. Солдаты, которые несут службу за свою страну на первом месте топа уважения.
Среднее образование. Колледжи учат сварщиков, плотников, электриков, кабельщиков. По американским меркам довольно недорого, с практически гарантированным трудоустройством на зарплату 5к-7к сразу после окончания.
Система публичных библиотек, в которых можно найти и заказать бесплатно практически любую книгу, даже изданную месяц назад. В условиях практически отсутствия пиратского контента, это крутецкая штука.
Общее ощущение
За три с лишним года жизни с США первое впечатление заместилось уже бытовым взглядом. Многое проявилось более явно. Этот абзац больше иррациональный и эмоциональный.
По известным причинам в США очень много денег. Они стекаются сюда со всего мира. Этим можно объяснить всю неэффективность бизнеса, растущие рынки с компаниями имеющими сроки окупаемости по 300 лет, и десятки стартапов, создающих стопятьсотый мессенджер. Интересно, сколько это может продолжаться?
Местное население живет в жиру (до последнего времени). Дешевые кредиты делают все доступным. Например, я прямо сейчас могу поехать в автосалон и купить новую Тойоту Короллу в кредит за 300 долл в месяц. Поэтому, когда в прошлом году хакеры взломали нефтепровод на нашем побережье и два дня были сложности с бензином (сложности, а не полное отсутствие), тут начался хаос с выносом продуктовых магазинов и драками на заправках. Причем в магазинах смели яйца и туалетную бумагу. Хз почему. Я боюсь представить что тут начнется, если разразится реальный кризис.
В последнее время на сайтах типа хабра или vc стала популярна точка зрения о том что "нужно валить". Не знаю. Если ты айтишник и едешь на зарплату 10-15к, то возможно. А если нет, то я бы крепко подумал.
"""

In [438]:
len(test_article)

21883

In [439]:
summarizer = LuhnSummarizer()

summary = summarizer.summarize(test_article)

len(test_article), len(summary)

(21883, 5616)

In [440]:
print(summary)


Два года назад я написал на хабре статью Как живется в США «неайтишникам», которая собрала 194 000 просмотров, 561 комментарий и вошла в список лучших статей 2020 года.И вот, два года спустя я решил пробежаться по тем же тезисам, чтобы провести некий срез, как изменилась обстановка в США и мое место в ней, особенно в свете последних событий.Обещаю без политики и максимально объективно.
!!!Все что я тут пишу это исключительно мой опыт и мое мнение.Я не претендую на истину в последней инстанции, а просто рассказываю как это было у меня.
Про мой язык
За два прошедших года мой Английский однозначно улучшился.Точнее даже не сам язык, а способность им пользоваться.Ушла закрепощенность и ощущение убогости.Теперь даже если я говорю неправильно, меня это не смущает.Причем все это в основном благодаря работе, где приходится очень много коммуницировать и читать огромное количество документации.Но тут же есть и обратная сторона.Окружающие меня американцы довольно свободно относятся к правилам гра

In [441]:
another_test_article = """
Желание перестраховаться – самое рискованное желание на свете.
Золотые наручники — когда ты остаешься на работе, которую предпочел бы бросить, только из-за денег.
Я думал, что буду работать в Netflix вечно. Самые высокие зарплаты по рынку. Свобода и уважение. Нелимитированные оплачиваемые нерабочие часы. Чего еще можно хотеть?
Поэтому, когда я ушел из Netflix в мае 21 года, все подумали, что я чокнулся. Родители первыми подняли бучу. Приехав из Китая, где им едва хватало еды, они думали, что я бросаю любую работу при возникновении каких-то сложностей, через которые им приходилось проходить, чтобы приехать в Америку.
“Просто не высовывайся и делай свою работу!” - говорили они.
“Будь благодарен за то, что у тебя есть!” - продолжали они возмущаться.
Никто из моих друзей также не мог в это поверить.
“А как же бесплатная еда?!”
”Ты работаешь в компании, которая входит в пятерку крупнейших IT-гигантов!" 
“Просто отдыхай и инвестируй, братан!”
Единственный аргумент, который я услышал против увольнения и который заставил меня немного задуматься, был от моего наставника в Netflix. Он сказал, что я не должен увольняться, не найдя другой работы, потому что так “я окажусь без рычага влияния, которым была моя высокая зарплата в Netflix”.
Отдых и инвестирование — ожидание размещения акций
Отдых и инвестирование — ожидание размещения акций
Это заставило меня сделать паузу на целых три дня, но потом я все равно бросил работу. Теперь, восемь месяцев спустя, я на 100% уверен, что это было самое правильное решение.
В своей статье я рассказываю про три фактора, которые помогли мне осознать реальную стоимость “золотых наручников” и понять, почему даже зарплата в полмиллиона долларов в год не смогла заставить меня остаться на работе, которая мне больше не нравилась.
Попытка смены ролей
С закрытием офисов в марте 2020 года все лучшее, что было в работе – общение, коллеги, льготы, – исчезло. А все, что осталось, – это сама работа. 
Поэтому, если изначально все, что у вас есть, – это работа, которая вам не нравится, COVID усилит эту ненависть в 10 раз.
Моя работа мне не нравилась. Но так было не всегда.
Я проработал в Netflix почти четыре года старшим инженером-программистом в отделе развития. Поначалу мне казалось, что мне слишком щедро платят за то, что я учусь. И на протяжении двух лет меня это устраивало. Культура Netflix сильно отличалась от, например, культуры в Amazon, которая была очень скрытной. Здесь памятки по каждому продукту доступны для ознакомления всем сотрудникам. Это было равнозначно тому, что тебе бы платили за получение степени MBA.
Прозрачность Netflix во всех своих продуктовых решениях была одним из лучших преимуществ работы там. Это контрастирует с более скрытной культурой Apple и Amazon.
Прозрачность Netflix во всех своих продуктовых решениях была одним из лучших преимуществ работы там. Это контрастирует с более скрытной культурой Apple и Amazon.
Тем не менее со временем начала казаться копипастом.
Нужно запустить новый микросервис?
Скопируйте и вставьте старый, измените бизнес-логику, и все готово.
Новый A/B тест?
Скопируйте и вставьте старый, измените несколько тестовых вариантов, и все готово.
Новый тест электронной почты?
Скопируйте и вставьте старый и… — я думаю, вы поняли.
У Netflix сильная культура A /B тестирования. Вот 4 варианта кнопки, которые они протестировали на главной странице (из технического блога Netflix)
У Netflix сильная культура A /B тестирования. Вот 4 варианта кнопки, которые они протестировали на главной странице (из технического блога Netflix)
Не было никаких сомнений, что инженерные разработки нужны в Netflix, но главный вопрос заключался в том – для какого именно продукта. Поэтому я захотел перейти в отдел управления продуктами, чтобы мои усилия были плодотворными. Я провел два года безостановочно общаясь со всеми отделами и подавая заявки на каждую должность, которую мог найти.
В какой бы отдел я ни приходил, я рассказывал, как расставлю приоритеты, если стану продукт менеджером: обслуживание клиентов, производительность разработчиков, создание студий, партнерские взаимоотношения и уведомления. Я предложил создать специальную позицию в моей команде, чтобы помогать управлять растущей инфраструктурой. Я также предположил, что другие PM могли бы делегировать мне больше своих задач, чтобы высвободить себе время для роста и развития своих отделов. Все эти предложения в конечном счете не увенчались успехом.
Я провел 2 года, ходя по кругу, пытаясь получить работу PM в Netflix, но в конечном итоге у меня закончились варианты.
Я провел 2 года, ходя по кругу, пытаясь получить работу PM в Netflix, но в конечном итоге у меня закончились варианты.
Оглядываясь назад, я осознаю свою ошибку. Я думал, если просто буду стараться усерднее, то в конце концов получу эту работу. Но теперь я понял, иногда вещи выходят из-под контроля из-за проблем в организационной структуре. У Netflix не налажены процессы для поддержки горизонтальных изменений ролей, подобных этому. Я никогда не сталкивался в этой компании с практикой, чтобы инженер успешно переходил на управление продуктами.
Мне предложили новые возможности и сотрудничество с отделом product management, чтобы я развивал навыки работы с продуктами, за что я, конечно, благодарен. Но сотрудничество – это не то же самое, что должность. В конечном счете, вы же не можете просто прочитать книгу о плавании и ожидать, что научитесь плавать. Вам надо залезть в воду.
Убывающая мотивация, убывающая производительность
Ближе к концу неудачных поисков работы на позиции PM я почувствовал, что высокая зарплата становится все более невыгодной сделкой. До этого я зарабатывал и учился. Теперь я только зарабатывал.
Цели моей команды и мои интересы также начали расходиться. Моя команда больше двигалась в направлении, более ориентированном на инженерные разработки, включая миграцию платформы. А мои интересы смещались в сторону предпринимательства и управления продуктами. Инженерная работа, порученная мне, не была более применима ни к какой будущей работе, которую мне придется выполнять
Стало казаться, что я снова совершаю предыдущую карьерную ошибку — остаюсь на работе, которая мне не очень подходит, дольше, чем следовало бы. Эта ошибка обходится дороже, чем думают люди. Если вы останетесь еще на 2 года на работе, которую хотите оставить, и сделаете это более, чем в 5 компаниях за свою жизнь, вы просто потратите впустую десять лет своей жизни, занимаясь тем, что вы не хотите делать. Я абсолютно точно ощущал, что зря трачу время.
Нет ничего более вызывающего беспокойство, чем пропущенные рабочие сообщения.
Нет ничего более вызывающего беспокойство, чем пропущенные рабочие сообщения.
Моя мотивация слабла, а вместе с ней падала и моя работоспособность. Я стал меньше участвовать в собраниях, свел к минимуму выполнение любой работы, которая не имела прямого отношения к развитию навыков управления продуктами, и начал избегать общения. Единственной мотивацией в конце концов стало стремление просто не быть уволенным. Было немного грустно осознавать, что наступил момент, когда моя планка так низко упала. И при этом я изо всех сил пытался пересечь даже ее.
К сожалению, мой менеджер начал замечать это. В ходе бурного обзора производительности, который длился более двух часов, он сказал, что мне следует 
1) быть более вовлеченным в миграцию платформы;
2) быть более общительным. 
По его словам, мне следовало совершенствоваться в этих областях, “если я хотел остаться в команде”.
Переоценка жизненных приоритетов во время COVID-19
Пандемия стала тревожным сигналом.
Наблюдение за тем, как миллионы людей умирают от COVID, заставило меня понять, что завтрашний день не гарантирован. Вы можете умереть от COVID еще до того, как осуществится какая-либо ваша мечта. И чем дольше вы ее откладываете, тем выше риск, что она никогда не сбудется. Поэтому, если есть что-то, чего вы хотите, необходимо решиться на это прямо сейчас.
Больше никаких “в следующий раз”. Сейчас самое время.
Отложенная мечта - это отвергнутая мечта. Моя любимая цитата всех времен.
Отложенная мечта - это отвергнутая мечта. Моя любимая цитата всех времен.
Я понял, какова реальная цена золотых наручников. Цена – ваша молодость, ваше время и ваша жизнь. Люди не совсем точно оценивают эти затраты, потому что зарплата – это твердая цифра, в то время как ценность молодости более неосязаема. Но только потому, что что-то трудно измерить, это не делает его менее ценным, чем что-то поддающееся учету, например, деньги. Трудно измерить ценность бренда, психического здоровья или любви, но мы знаем, что это важно.
Видя, как люди умирают от COVID, я испугался, что однажды на моем надгробии будет написано:
“Здесь лежит Майкл. Он провел свою жизнь, выполняя работу, которой никогда не хотел заниматься. Потом он подхватил COVID и умер. Покойся с миром”.
Чем дольше я оставался на работе, которая мне не нравилась, тем выше становились шансы, что именно эта надпись окажется на моем надгробии. Я знал, что должен действовать сейчас — я не мог продолжать откладывать сложные вопросы, связанные с карьерой, на потом. Мне пришлось уволиться.
Последние дни
Я рассматривал плохую оценку производительности и угрозу увольнения как выход из положения. Но я хотел сначала получить выходное пособие, не будучи уволенным.
Поэтому я сделал предложение своему менеджеру на one-to-one спустя несколько недель после того, как мы обсуждали выходное пособие.
Я сказал что-то вроде: “Моя производительность снижается, потому что моя мотивация падает. Я не вижу вариантов, как ее улучшить, потому что цели команды все больше расходятся с моими карьерными целями. Что, если мы просто обсудим выходное пособие от Netflix сейчас, а не будем затягивать это? Таким образом, Netflix сэкономит деньги, вы сможете быстрее найти более подходящего специалиста, и я смогу начать делать то, что хочу. Беспроигрышное решение для всех”.
Освобождение
Освобождение
После того, как он обсудил это с отделом кадров, у меня была последняя общая встреча с ними, на которой меня согласились досрочно уволить, и я получил свое окончательное выходное пособие. С золотыми наручниками было покончено.
Жизнь после Netflix
Я думал, что моя жизнь закончится после ухода из корпорации, но все оказалось с точностью до наоборот. Я беспокоился, что у меня не будет социальной жизни, но на самом деле после увольнения я встретил еще больше людей — создателей, предпринимателей и строителей.
Я увидел, что мое психическое здоровье улучшилось, поскольку состояние тревоги, возникшее у меня из-за постоянных беспокойств о том, что я пропустил очередной мейл или сообщение, исчезла.
Теперь я ощущаю глубокое спокойствие внутри, непоколебимую веру, что все будет хорошо, даже если какой-либо будущий успех прямо сейчас не гарантирован. То, что я печатаю этот текст в воскресенье вечером, означает, что у меня нет проблем с переработками по выходным. Нет лучшего стимула, чем осознание ценности своей собственной работы.
Работая только с проектами, которые заряжают энергией, я осознал иронию судьбы, поскольку начал получать гораздо более высокий доход, даже в сравнении с прошлым местом.
Прошло восемь месяцев с тех пор, как я уволился в мае 2021 года. Я взял перерыв до конца 2021 года. Я прожил в Нью-Йорке несколько месяцев, совершил поездку по Юте и Аризоне, где просто наслаждался жизнью.
Я решил полностью посвятить себя работе на самого себя. Хотя я только начинаю и у меня нет никаких реальных надежных источников дохода, я считаю, что, если буду работать над тем, что заряжает меня энергией, произойдут хорошие вещи.
Теперь я искренне верю, что перестраховаться – самый рискованный выбор из всех возможных. Когда вы играете безопасно, вы так же подвержены всем опасностям, за исключением того, что у вас нет никаких шансов на положительный результат. Как однажды сказала Хелен Келлер:
Перестраховаться - самый рискованный выбор из всех возможных.
"""

In [442]:
summarizer = LuhnSummarizer()

summary = summarizer.summarize(another_test_article)

print(len(another_test_article), len(summary))

11828 2408


In [443]:
print(summary)


Желание перестраховаться – самое рискованное желание на свете.
Золотые наручники — когда ты остаешься на работе, которую предпочел бы бросить, только из-за денег.
Я думал, что буду работать в Netflix вечно.Самые высокие зарплаты по рынку.Свобода и уважение.Нелимитированные оплачиваемые нерабочие часы.Чего еще можно хотеть?
Поэтому, когда я ушел из Netflix в мае 21 года, все подумали, что я чокнулся.Родители первыми подняли бучу.Приехав из Китая, где им едва хватало еды, они думали, что я бросаю любую работу при возникновении каких-то сложностей, через которые им приходилось проходить, чтобы приехать в Америку.
“Просто не высовывайся и делай свою работу!” - говорили они.
“Будь благодарен за то, что у тебя есть!” - продолжали они возмущаться.
Никто из моих друзей также не мог в это поверить.
“А как же бесплатная еда?!”
”Ты работаешь в компании, которая входит в пятерку крупнейших IT-гигантов!"
“Просто отдыхай и инвестируй, братан!”
Единственный аргумент, который я услышал против увольне